<a href="https://colab.research.google.com/github/RobinHamers/mombasa_solutions/blob/main/create_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai import types
from google.genai.types import (
    HttpOptions,
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)
import os
import pandas as pd
import geopandas as gpd
import sys
from pydantic import BaseModel
from typing import List

In [ ]:
OUTPUT_RISK_EXPLAIN_JSON = '{"fire": ["string"], "explanation_fire": ["string"], "heat": ["string"], "explanation_heat": ["string"], "flood": ["string"], "explanation_flood": ["string"]}'
class RiskActions_and_explanation(BaseModel):
    fire:List[str]
    explanation_fire:List[str]
    heat:List[str]
    explanation_heat:List[str]
    flood:List[str]
    explanation_flood:List[str]

In [ ]:
OUTPUT_RISK_EXPLAIN = """
    Output the solutions and explain (chain of thoughts) why you propose this solution refering to the inputs you considered.
    Format: 2 times 3 sections: 🔥 Fire, 🌡 Heat, 🌊 Flood. Max 3 actions each. Only if risk is relevant.
    Each solution = 1 bullet, 1–2 lines. explanation for that solution in its seperate section also following the bullets
    """

In [ ]:
 system_instruction = f"""
    You are an environmental expert creating actionable mitigation recommendations for climate and disaster resilience in urban settings.

    Below is a description of the current risks and environmental conditions for a municipality and a specific local zone. You are tasked with generating mitigation actions tailored to that zone's risks, demographic context, and relevant points of interest.

    Use clear language and address actions that can be taken both by individuals and by local authorities. Use bullet points where helpful. Be specific, not generic.
    When referring to community members, use 'community members at-risk' instead of terms like 'elderly' or 'vulnerable' that may not be appropriate in all contexts.

    🎯 TASK:
    Based on this information, list **concise practical and location-specific mitigation actions** that can reduce climate and disaster risks in the local area. Structure them by risk type (e.g., Heat, Flood, Fire). Include targeted suggestions related to nearby POIs or vulnerable populations.

    {OUTPUT_RISK_EXPLAIN}
    """

In [ ]:
def prepare_prompt_risk_action(municipality_context, heat_risk, flood_risk, fire_risk, lst_day, lst_night,
                          sealed_surface_pct, canopy_cover_pct, elevation, river_proximity,
                          flood_plain, tree_count, flammability, tree_connectivity,
                          fire_history_info, population_density, vulnerable_groups, pois, climate_driven_impassable_roads,
                          emergency_assemble_areas, comments, pdf_uri=None, synthesis_pdf=None, cache=None, print_output=False
                          ):
    prompt = f"""

            INPUT
            ---
            🟩 GENERAL CONTEXT (Municipality-Level):
            # {municipality_context}

            🟨 LOCAL CONTEXT (Buffer zone of ~500m radius):
            - Heat Risk (monthly): {heat_risk}
            - Flood Risk: {flood_risk}
            - Fire Risk (monthly): {fire_risk}
            - LST Day/Night: {lst_day}/{lst_night}
            - Land Surface: Sealed Surface = {sealed_surface_pct}%
            - Canopy Cover = {canopy_cover_pct}%
            - Elevation = {elevation}m
            - River proximity = {river_proximity} m
            - Floodplain: {flood_plain}
            - Tree cover: Number of trees = {tree_count}
            - Flammability Index = {flammability}
            - Connectivity = {tree_connectivity}
            - Fire history: {fire_history_info}


            📍 POINTS OF INTEREST:
            - Nearby: {pois}
            - climate_driven_impassable_roads: {climate_driven_impassable_roads}
            - emergency_assemble_areas: {emergency_assemble_areas}
            ---

            COMMENTS:
            {comments}
        """
    return prompt

In [ ]:
PROMPT_TEMPLATE = prepare_prompt_risk_action(
            municipality_context=municipality_context,
            heat_risk=heat_risk,
            flood_risk=flood_risk,
            fire_risk=fire_risk,
            lst_day=lst_day,
            lst_night=lst_night,
            sealed_surface_pct=sealed_surface_pct,
            canopy_cover_pct=canopy_cover_pct,
            elevation=elevation,
            river_proximity=river_proximity,
            flood_plain=flood_plain,
            tree_count=tree_count,
            flammability=flammability,
            tree_connectivity=tree_connectivity,
            fire_history_info=fire_history_info,
            population_density=population_density,
            vulnerable_groups=vulnerable_groups,
            pois=pois,
            climate_driven_impassable_roads=climate_driven_impassable_roads,
            emergency_assemble_areas=emergency_assemble_areas,
            comments=comments,
            pdf_uri=pdf_uri, #synthesis_pdf, #pdf_uri, # synthesis_pdf, # cache=cached_files,
            cache=cache,
            print_output=print_output
        )